__<span style="background-color:#ffffe0">第3章　時系列予測モデル構築・超入門</span>__

# 3.4　季節成分が複数ある場合の予測モデル

## 3.4.4　時系列特徴量を生成しテーブルデータを作ろう！

### 生成する時系列特徴量

* ラグ特徴量（ラグ1）：lag1
* ローリング特徴量（1期前までの1日間の平均の平均）：window48
* エクスパンディング特徴量（1期前までの平均）：expanding
* 三角関数特徴量（48周期）：sin48, cos48
* 三角関数特徴量（336周期）：sin336, cos336

### 準備（必要なモジュールとデータの読み込み）

In [1]:
#
# 必要なモジュールの読み込み
#

import numpy as np
import pandas as pd

import statsmodels.api as sm

import warnings
warnings.simplefilter('ignore')

In [2]:
#
# 必要なデータセット（時系列データ）の読み込み
#

dataset = sm.datasets.get_rdataset(
    "taylor", "forecast")
y = dataset.data

print(y) #確認

          x
0     22262
1     21756
2     22247
3     22759
4     22549
...     ...
4027  27946
4028  27133
4029  25996
4030  24610
4031  23132

[4032 rows x 1 columns]


### ラグ特徴量・ローリング特徴量・エクスパディング特徴量

In [3]:
#
# ラグ特徴量・ローリング特徴量・エクスパディング特徴量
#

# ラグ特徴量（ラグ1）の生成
lag1 = y.shift(1)

# ローリング特徴量（1期前までの1日間の平均）の生成
window48 = lag1.rolling(window=48).mean()

# エクスパンディング特徴量（1期前までの平均）の生成
expanding = lag1.expanding().mean()

In [4]:
#
# 作成した時系列特徴量を結合しテーブルデータを生成
#

## データを結合
df_tbl = pd.concat([y,
                    lag1,
                    window48,
                    expanding],
                   axis=1)

## 変数名を設定
df_tbl.columns = ['y',
                  'lag1',
                  'window48',
                  'expanding']

print(df_tbl) #確認

          y     lag1      window48     expanding
0     22262      NaN           NaN           NaN
1     21756  22262.0           NaN  22262.000000
2     22247  21756.0           NaN  22009.000000
3     22759  22247.0           NaN  22088.333333
4     22549  22759.0           NaN  22256.000000
...     ...      ...           ...           ...
4027  27946  28677.0  25060.208333  29621.921033
4028  27133  27946.0  25046.520833  29621.504965
4029  25996  27133.0  25034.812500  29620.887317
4030  24610  25996.0  25019.125000  29619.987841
4031  23132  24610.0  25003.041667  29618.744976

[4032 rows x 4 columns]


### 三角関数特徴量

In [5]:
#
# 三角関数特徴量
#

# 空のデータフレーム生成
exog = pd.DataFrame()
exog.index = y.index

# 三角関数特徴量（Fourier terms）の生成関数
def fourier_terms_gen(seasonal,terms_num):
    
    #seasonal:周期
    #terms_num:Fourier termの数（sinとcosのセット数）
    
    for num in range(terms_num):
        num = num + 1
        sin_colname = 'sin'+str(seasonal)+'_'+ str(num)
        cos_colname = 'cos'+str(seasonal)+'_'+ str(num)
        exog[sin_colname] = np.sin(num * 2 * np.pi * exog.index / seasonal)
        exog[cos_colname] = np.cos(num * 2 * np.pi * exog.index / seasonal)
        
# 三角関数特徴量の生成
## 336周期
fourier_terms_gen(
    seasonal=336,
    terms_num=10)

## 48周期
fourier_terms_gen(
    seasonal=48,
    terms_num=10)

print(exog) #確認

      sin336_1  cos336_1  sin336_2  cos336_2  sin336_3  cos336_3  sin336_4   
0     0.000000  1.000000  0.000000  1.000000  0.000000  1.000000  0.000000  \
1     0.018699  0.999825  0.037391  0.999301  0.056070  0.998427  0.074730   
2     0.037391  0.999301  0.074730  0.997204  0.111964  0.993712  0.149042   
3     0.056070  0.998427  0.111964  0.993712  0.167506  0.985871  0.222521   
4     0.074730  0.997204  0.149042  0.988831  0.222521  0.974928  0.294755   
...        ...       ...       ...       ...       ...       ...       ...   
4027 -0.093364  0.995632 -0.185912  0.982566 -0.276836  0.960917 -0.365341   
4028 -0.074730  0.997204 -0.149042  0.988831 -0.222521  0.974928 -0.294755   
4029 -0.056070  0.998427 -0.111964  0.993712 -0.167506  0.985871 -0.222521   
4030 -0.037391  0.999301 -0.074730  0.997204 -0.111964  0.993712 -0.149042   
4031 -0.018699  0.999825 -0.037391  0.999301 -0.056070  0.998427 -0.074730   

      cos336_4  sin336_5  cos336_5  ...       sin48_6       cos

### 2つのテーブルデータの結合と欠測値削除

In [6]:
#
# 2つのテーブルデータの結合と欠測値削除
#

# データを結合
df_tbl = pd.concat([df_tbl,
                    exog],
                   axis=1)

# 欠測値削除
df_tbl = df_tbl.dropna()

print(df_tbl) #確認

          y     lag1      window48     expanding  sin336_1  cos336_1   
48    25093  26572.0  31398.145833  31398.145833  0.781831  0.623490  \
49    24320  25093.0  31457.125000  31269.469388  0.793353  0.608761   
50    24684  24320.0  31510.541667  31130.480000  0.804598  0.593820   
51    25338  24684.0  31561.312500  31004.078431  0.815561  0.578671   
52    24943  25338.0  31615.041667  30895.115385  0.826239  0.563320   
...     ...      ...           ...           ...       ...       ...   
4027  27946  28677.0  25060.208333  29621.921033 -0.093364  0.995632   
4028  27133  27946.0  25046.520833  29621.504965 -0.074730  0.997204   
4029  25996  27133.0  25034.812500  29620.887317 -0.056070  0.998427   
4030  24610  25996.0  25019.125000  29619.987841 -0.037391  0.999301   
4031  23132  24610.0  25003.041667  29618.744976 -0.018699  0.999825   

      sin336_2  cos336_2  sin336_3  cos336_3  ...       sin48_6       cos48_6   
48    0.974928 -0.222521  0.433884 -0.900969  ... -1.4

### CSVファイルとして出力

In [7]:
df_tbl.to_csv('taylor_tbl_.csv', index=False)